VGG 16'yı kodlayalım.Makaledeki D'yi yapıyoruz.

In [ ]:
from keras.models import Sequential,Model
from keras.layers import Conv2D,MaxPooling2D,ZeroPadding2D,Flatten,Dense,Dropout,Input
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
#Shape diyerek girdi görüntüsünün boyutunu veriyoruz.Makalede 224,224,3'lük bir görüntüden bahsetmiş.
#Makalede hepsinin paddingini 1 aldığından bahsetmiş.
#3x3'lük filtreler var, bazı konvolüsyon katmanlarıda 1e1'lik filtreler kullandığından
#bahsetmiş.Max pooling yapmış 2,2 stride'ı 2 olduğundan bahsetmiş.Mimari detayları var.


input = Input(shape=(112,112,3))


#İlk katmanda 64,64 diye peşpeşe 2 tane yapı var.
#Stride 1e1 herhangi bir şey değiştirmiyoruz.
#Paddingi same olabilir çünkü genelde outputlar çok değişmiyor.
#Görselde konvolüsyon katmanlarında model değişiklik yapmamış.Görsel notta var.
#Değişiklik olmayacaksa padding same diyeceğiz.Çünkü konvolüsyon katmanlarında herhangi bir
#daralma,küçülme olmayacak.


x=Conv2D(64,kernel_size=(3,3),padding='same',activation='relu')(input)

x=Conv2D(64,kernel_size=(3,3),padding='same',activation='relu')(x)

x=MaxPooling2D(pool_size=(2,2),strides=(2,2))(x)


#2.bloğa geçelim.


#Konvolüsyonel katmanların hepsine paddingi same yaptık çünkü Vgg mimariye bakarrsak;
#daralmalar sadece max pooling katmanlarında oluyor.Konvolüsyonel katmanlarda girdi boyutu
#hep standart olarak kalmış.



x = Flatten()(x)

x = Dense(4096,activation='relu')(x)

x = Dense(4096,activation='relu')(x)

output = Dense(1000,activation='softmax')(x)

vgg16 = Model(inputs=input,outputs=output)

#Artık Vgg16 modelimizi oluşturmuş olduk.
#Özet görelim.138 milyon parametresi var.

vgg16.summary()





Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 224, 224, 64)      1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 224, 224, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 64)     0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 112, 112, 128)     73856     
                                                                 
 conv2d_3 (Conv2D)           (None, 112, 112, 128)     147584    
                                                             

In [ ]:
#Modeli kodlamasaydık ağırlıklarını nereden indirecektik.
#Transfer Learning yapmak istiyorum.Bu modeli başka bir problemde kullanmak istiyorum ama
#imagenet ağırlıklarınıda almak istiyoruz.
#Keras pre-trained yazarsak, keras içerisinde bizim için eğitilmiş modeller vardır.Bu modellerin hepsini kendi problemimize uyarlayıp eğitebiliriz.
#Bunların ağırlıkları burada saklanır.İstediğimiz zaman alabiliriz.
#Örneğin VGG16 138 milyon parametresi var. İmageNet üzerinde top5 hata oranı %90.1, top1 ise
#%71 miş. 528 mb olduğundan bahsediliyor.Çok büyük bir moel.Derinliği 16.
#Vgg çok büyük bir model.Eğitmesi kolay değil.

#Bunun haricinde farklı mimariler var.Geliştirilen mimariler imagenette eğitilmiş.
#Eğitildikten sonrada performasnlarına göre sıralanmış.


#Peki biz bu modelleri kendi uygulamamız içerisinde nasıl alabiliriz.Nasıl tahmin ettirebiliriz.
#Extrat features with VGG16 demiş.Burada napabiliriz:Bir görüntü verip bu görüntüye VGG'nin
#tahmin üretmesini sağlayabiliriz.
#Ya da katmanları kendimiz değiştirebiliriz.


#Extract features with VGG16 kısmına bakıyoruz
#Vgg'yi import etmiş.
#Vgg ile ilgili bir deneme yapacağız.İmagenetten bir değer verip üretmeye çalışacağız.

#Eğitilmiş bir Vgg modeline bir görüntü vereceğiz.Bu görüntü üzerinde bakalım tahmin yapabilecekmi.


In [ ]:
#Şu an direk oradan hazır modeli alıp buraya getircez.Ağırlıklar falan her şey öğrenilmiş.
#Hazır bir model.Sınıflandırma yapmaya hazır.Elimizdeki modeli resimi kullanalım.



from tensorflow.keras.preprocessing.image import load_img,img_to_array
import numpy as np
from keras.applications import VGG16

#weiights: kernellerin ağırlıklarını nerden alayım diyor.
#Bizde diyoruzki imagenet ağırlıklarından alabilirsin.
#include_top: Modelin sınıflandırıcı kısmını istiyor musun diye soruyor.
#Bir cnn mimarisi konvolüsyon(feature çıkarma katmanı) ve sınıflandırıcı katmanlarınadn oluşur.
#Yani imagenetin sınıflandırıcısınıda getireyim mi diyor.Biz dedikki,sınıflandırıcı
#değiştirmemiz lazım, farklı problem için kullanacaksak en azından çıktısıyla oynamamız gerekiyor.
#O yüzden mutlaka burayı değiştirmemiz gerekiyor.Şu an farkı göstermek amaçlı True dedik.


#target size verebiliyoruz.Yani hangi boyutta okuyim diyor.Bizde diyoruzki benim VGG
#224 alıyor, buna göre veriyi oku.
img = load_img('tank.jpg',target_size=(224,224))


#Veriyi jpg türünde okuduk.Numpy arraye çevirmemiz lazım.
img = img_to_array(img)


#Kaç tane veriyi vereceğimizi söylemek için bir reshape işlemi yapalım.
#İlk verdiğimiz 1 1 görüntü vereceğimiz için 1.

#image'nin shapeini görelimki neden img.shape[0] aldık bakalım.
print(img.shape)

img=img.reshape((1,img.shape[0],img.shape[1],img.shape[2]))

tf_vgg16 = VGG16(weights='imagenet',include_top=True)

tf_vgg16.summary()

#VGG16

#Decode predictions, tahminlere decode etmek için bu fonksiyona ihtiyacımız var.
from keras.applications.vgg16 import preprocess_input,decode_predictions

img = preprocess_input(img)

#Hiç eğitmeden predictionu verebiliyoruz.
pred = tf_vgg16.predict(img)


#Nasıl predictionlar geliyor bir görelim.
#pred            #buna bakıp sildik.

#224,224,3 görüntüyü yükledi.Sonrasında reshape etti.Vgg16'ya tahmin etmek için gödnerdik.
#Bir sürü olasılık verdi bize.1000'li softmax kullandığı için.

#Hangi sınıfa ait bir tahmin yaptığını görebilmek için;
#resulta bakalım.result en yüksek değeri vermesi lazım.
result = np.argmax(pred,axis=-1)
result

#result array[278] verdi.Yani 278 nolu sınıfa ait diyor.Yani imagenette 278 ne bilmiyoruz
#o yüzden burada decode_predictions var bunu kullanabiliriz.

label = decode_predictions(pred)

print(label)

#top 5 gibi bir değer verdi.En yüksek 5 tahmini verdi.
#Burada şunu yapmış olduk.Bir Vgg modelimizin, herhangi bir görüntü için tahmin yapmış olmasını sağladık.


In [ ]:
#Biz bu modeli kullandık ama bu model eğitilmiş bir model.Görüntüleri verip bunu kullanabiliyoruz.
#Peki bu modeli kendi verimizle eğitmek istesek ne yapmalıyız ona bakalım.

#Yani Vgg'yi aldık ama vggnin sadece konvolüsyonel katmanlarındaki ağırlıklarına ihtiyacımz var.
#Diğer katmanlarındaki ağırlıklara ihtiyacı yok.O zaman ne yapacağız.
#Kendi problemimiz için bunu düzenleyeceğiz.
#Bunun içinde include_top'u False yaparak başlıyoruz.
#İnclude_top false demek sınıflandırıcı katmanınını getirme demek.



In [ ]:
#base_model = VGG16(weights='imagenet',include_top=False,input_shape=(224,224,3))
#tf_vgg16.summary()

#Geldi geldi 7,7,512lik feature extraction kısmında bıraktı.
#14 milyon parametreye düştü.138 milyondan 14milyon parametreye düştü.
#Bunun anlamı: Şunu kesin gözlemlememiz lazım. Bunun esas ana kısmını kim oluşturuyor.
#Yani 138 milyon parametrelik kısmını neredeyse 124 milyon parametrelik kısmını sınıflandırıcı
#kısmından geldiğini görüyoruz.Yani tam bağımlı katmanların olduğu taraftan geldiğini görüyoruz.

#O yüzden problemimizde 1.2 milyon görüntümüz yoksa bu kadar tam bağımlı katmana ihtiyacımız yok.
#Bu kadar parametreyede ihtiyacımız olmayabilir.
#Ne yapacağız.İmagenetten konvolüsyonel katmanları aldık imagenetten.Sınıflandırıcı katmanları almadık.
#Sınıflandırıcı katmanları kendimiz modifiye ederek oluşturalım.

base_model = VGG16(weights='imagenet',include_top=False,input_shape=(224,224,3))


#False dersek, top kısmını aldığımız konvolüsyonel katmanlar eğitime katılmayacak.
#Yalnızca sınıflandırıcıyı eğiteceğiz.
#x
base_model.trainable = False


#Flattene kadar geldik.Veriyi flat edelim.Base modelin outputu üstteki çıktıdaki flatten yazann satırda 25088
#lik olan kısım.25088'i flat edeceğiz.

x=Flatten()(base_model.output)

#Dense katmanı ekleyelim.
x = Dense(64,activation='relu')(x)

#Pneumonia sınıflandıracağımız için çıktı katmanımız 1 olacak.
output = Dense(1,activation='sigmoid')(x)


#VGG'yi base alan Modelimiz hazır.

new_model = Model(inputs=base_model.inputs,outputs=output)

new_model.summary()


#Burada ne yapmış olduk:VGG'nin include false olan modelini aldık.Kendimiz feature katmanını flat ettik.
#Daha sonra bir tane dense katmanı ekledik.Sonra bir dense katmanıyla çıktıyla bağladık.
#Buraya dense katmanlarını kendimiz değiştirebiliriz.

#64lük dense ve 1 çıktı eklediğimizden 16 milyonluk parametreli bir model oluşturmuş olduk.


#Hangi katmanlar eğitilecek hangi katmanlar eğitilmeyecek.Bunlarla ilgili bir ayarlama yapmamız gerekiyor.
#Biz artık modeli pneumonia verimiz için eğitime sokabiliriz.

#Şunu görmek istiyoruz: Hangi katmanlar eğitilebilir durumda.Bunu görelim.
#Tüm katmanlar için bir çıktı verecek.

for layer in new_model.layers:
  print(layer.name," ",layer.trainable)


#Hepsi true geldi.Anlamı şu.Bu modeli aldık.İmagenet ağırlıklarıyla yükledik ama sonunuda yani sınıflandırıcı
#katmanı kendi problemimize göre değiştirdik.Eğitime başlarsan buradaki bütün filtreleri baştan itibaren
#güncelleyeceksin, tekrar eğitime sokacaksın anlamına geliyor.
#Bunu istemiyoruz.Yani base modelin eğitilmiş olmasını istemiyoruz.Sadece sınıflandırıcıyı eğitmek istiyoruz.
#Yaptığımızı yukarıda x ile gösterelim.

#False yaptıktan sonra konvolüsyon katmanların yani feature çıkaracak katmanların hepsi false olmuş
#Yalnızca Flatten ve son 2 katman eğitilebilir.
#Parametre 16 milyon.Trainable parametre 1.6 milyon.
#Konvolüsyonel katmanların hiçbirini eğitmiyoruz.Yalnızca problemimize göre buradaki dense katmanlarımızı eğitmiş
#oluyoruz.



#Peki bazı katmanların bir kısmını eğitebilsek geri kalan kısımları eğitmesek.
#Onu alta yapalımki kafamız karışmasın.


#Çok önemli.Hep ilk katmanlardan itibaren eğitime katmamaya çalışıyoruz neden?
#Çünkü ilk katmanlar veriye has özellikler değilde, genel özellikler ortaya çıkardığı için biz her halükarda hangi veri
#seti kullanırsak kullanalım genel özelliklere yine ihtiyacımız olacak.
#O yüzden ilk katmanlardan yani girdiye yakın olan katmanlardan itibaren dondurmaya başlıyoruz.



In [ ]:
base_model = VGG16(weights='imagenet',include_top=False,input_shape=(224,224,3))

#Böyle yaparsak base modeldeki tüm katmanları false yaparız.
#for layer in base_model.layers:
#  layer.trainable = False

#Bir kısmını eleyeceğiz.O yüzden şöyle yapalım.Baştan itibaren 5 tanesini alalım.
#Bu sayede ilk 5 katman false oldu.
#Pooling bloklarında train edilebilecek parametre olmadığı için 5'i 8 yaparsak
#7 tane katmanımızı,konvolüsyonel bloğumuzu false edecektir.
#Yani baştan itibaren bir kısmını eğitime katmadık.Geri kalanları eğitmiş olduk.
for layer in base_model.layers[:5]:
  layer.trainable = False

x=Flatten()(base_model.output)

x = Dense(64,activation='relu')(x)

output = Dense(1,activation='sigmoid')(x)

new_model = Model(inputs=base_model.inputs,outputs=output)

new_model.summary()

for layer in new_model.layers:
  print(layer.name," ",layer.trainable)

new_model.compile(loss='binary_crossentropy',optimizer="sgd",metrics="accuracy")

#Modeli derledikten sonra modeli eğitmemiz lazım.

#Alttaki trainData ve ValDatayı alamadığımızdan okuyamıyor.O drivedan veriyi okuduğumuz kodlar üstte olacak.
history=new_model.fit(trainData,epochs=15,validation_data=valData)


Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_17 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

NameError: ignored

In [ ]:
#Şimdi eğitime koyalım.

#0'dan oluşturduğumuzla hazır aldığımızın arasındaki fark şu: 0'dan oluşturduğumuz ağırlıklarını
#0'dan atıyor.Ama hazır aldığımız VGG16 modeli aynı vgg ama ağırlıklarını imagenetten alıyor.
#Yani eğitilmiş bir modelle hiç eğitilmemiş bir model alıyoruz.İkiside aynı mimariye sahip.


train_path = './drive/MyDrive/Deep Learning/chest_xray/train'
test_path = './drive/MyDrive/Deep Learning/chest_xray/test'

trainGen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=45,
    vertical_flip=True,
    horizontal_flip=True,
    zoom_range=0.5,
    validation_split = 0.1
)

valGen=ImageDataGenerator(rescale=1./255)
testGen=ImageDataGenerator(rescale=1./255)

trainData=trainGen.flow_from_directory(
    train_path,
    target_size=(224,224),
    class_mode='binary',
    batch_size=8,
    color_mode='grayscale',
    subset='training'
)

valData=trainGen.flow_from_directory(
    train_path,
    target_size=(224,224),
    class_mode='binary',
    batch_size=8,
    color_mode='grayscale',
    subset='validation'
)

testData=testGen.flow_from_directory(
    test_path,
    #test datayı 28 28 yapmıştık çok bozuk geldi görüntü.
    target_size=(224,224),
    class_mode='binary',
    color_mode='grayscale',
    batch_size=1
)

x,y=testData[602]
print(y.shape)
print(y)

#Class indislerini göster dediğimizde, normale 0 indisini atmış hastaya 1 indisini.
#Alfabetik olarak atıyor.
print(testData.class_indices)

#Test verisindeki 0 ve 1 etiketlerini görüyoruz.
print(testData.classes)
print(testData.class_indices)
print(testData.class_mode)

for i in range(6):
  img,y=testData.next()
  plt.subplot(2,3,i+1)
  print(img.shape)
  plt.imshow(img[0])
  plt.title(y[0])
  plt.axis('off')

plt.show()

